# MemoryVectorStore

MemoryVectorStore is an in-memory, ephemeral vectorstore that stores embeddings in-memory and does an exact, linear search for the most similar embeddings. The default similarity metric is cosine similarity, but can be changed to any of the similarity metrics supported by [ml-distance](https://mljs.github.io/distance/modules/similarity.html).


## Setup

To access the in memory vector store you'll need to install the `langchain` integration package.

```{=mdx}
import IntegrationInstallTooltip from "@mdx_components/integration_install_tooltip.mdx";
import Npm2Yarn from "@theme/Npm2Yarn";

<IntegrationInstallTooltip></IntegrationInstallTooltip>

<Npm2Yarn>
  langchain
</Npm2Yarn>
```

### Credentials

If you want to get automated tracing of your model calls you can also set your [LangSmith](https://docs.smith.langchain.com/) API key by uncommenting below:

```typescript
// process.env.LANGCHAIN_TRACING_V2="true"
// process.env.LANGCHAIN_API_KEY="your-api-key"
```

## Instantiation

In this example we'll use OpenAIEmbeddings, however you can use any embeddings you like.

In [2]:
import { MemoryVectorStore } from 'langchain/vectorstores/memory'
import { OpenAIEmbeddings } from '@langchain/openai'

const vectorStore = new MemoryVectorStore(new OpenAIEmbeddings())

## Manage vector store

### Add items to vector store

In [4]:
import { Document } from "@langchain/core/documents";

const document1 = new Document({
  pageContent: "foo",
  metadata: { source: "https://example.com" }
});

const document2 = new Document({
  pageContent: "bar",
  metadata: { source: "https://example.com" }
});

const document3 = new Document({
  pageContent: "baz",
  metadata: { source: "https://example.com" }
});

const documents = [document1, document2, document3];

await vectorStore.addDocuments(documents);

## Query vector store

Once your vector store has been created and the relevant documents have been added you will most likely wish to query it during the running of your chain or agent. 

### Query directly

Performing a simple similarity search can be done as follows:

In [9]:
const similaritySearchResults = await vectorStore.similaritySearch("thud", 1, (doc: Document) => doc.metadata.source === "https://example.com")

for (const doc of similaritySearchResults) {
  console.log(`* ${doc.pageContent} [${JSON.stringify(doc.metadata, null)}]`);
}

* foo [{"source":"https://example.com"}]


If you want to execute a similarity search and receive the corresponding scores you can run:

In [10]:
const similaritySearchWithScoreResults = await vectorStore.similaritySearchWithScore("thud", 1, (doc: Document) => doc.metadata.source === "https://example.com")

for (const [doc, score] of similaritySearchWithScoreResults) {
  console.log(`* [SIM=${score.toFixed(3)}] ${doc.pageContent} [${JSON.stringify(doc.metadata, null)}]`);
}

* [SIM=0.804] foo [{"source":"https://example.com"}]


### Query by turning into retriever

You can also transform the vector store into a retriever for easier usage in your chains. 

In [11]:
const retriever = vectorStore.asRetriever()
await retriever.invoke("thud")

[
  Document {
    pageContent: 'foo',
    metadata: { source: 'https://example.com' },
    id: undefined
  },
  Document {
    pageContent: 'baz',
    metadata: { source: 'https://example.com' },
    id: undefined
  },
  Document {
    pageContent: 'bar',
    metadata: { source: 'https://example.com' },
    id: undefined
  }
]


Using retriever in a simple RAG chain:

First, select the LLM you'd like to use:

```{=mdx}
import ChatModelTabs from "@theme/ChatModelTabs";

<ChatModelTabs customVarName="llm" />
```

In [12]:
// @lc-docs-hide-cell
import { ChatOpenAI } from "@langchain/openai";

const llm = new ChatOpenAI({ modelName: "gpt-4o-mini", temperature: 0, });

In [14]:
import { pull } from "langchain/hub";
import { StringOutputParser } from "@langchain/core/output_parsers";
import { RunnablePassthrough, RunnableSequence } from "@langchain/core/runnables";
import { ChatPromptTemplate } from "@langchain/core/prompts";

const prompt = await pull<ChatPromptTemplate>("rlm/rag-prompt");

const formatDocs = (docs: any[]) => {
  return docs.map(doc => doc.pageContent).join("\n\n");
};

const ragChain = RunnableSequence.from([
  {
    context: (input: string) => retriever.invoke(input).then(formatDocs),
    question: new RunnablePassthrough()
  },
  prompt,
  llm,
  new StringOutputParser()
])

await ragChain.invoke("What does the context say?");

The context includes three terms: "foo," "baz," and "bar." These terms appear to be listed without additional information or explanation. Therefore, the context does not provide any specific details or meaning.


## Create a new index from a loader

In [16]:
import { MemoryVectorStore } from "langchain/vectorstores/memory";
import { OpenAIEmbeddings } from "@langchain/openai";
import { TextLoader } from "langchain/document_loaders/fs/text";

// Create docs with a loader
const customLoader = new TextLoader("../../../../../examples/src/document_loaders/example_data/example.txt");
const customLoaderDocs = await customLoader.load();

// Load the docs into the vector store
const vectorStoreFromCustomLoader = await MemoryVectorStore.fromDocuments(
  customLoaderDocs,
  new OpenAIEmbeddings()
);

// Search for the most similar document
const resultFromCustomLoader = await vectorStoreFromCustomLoader.similaritySearch("hello world", 1);

console.log(resultFromCustomLoader);

[
  Document {
    pageContent: 'Foo\nBar\nBaz\n\n',
    metadata: {
      source: '../../../../../examples/src/document_loaders/example_data/example.txt'
    },
    id: undefined
  }
]


## API reference

For detailed documentation of all MemoryVectorStore features and configurations head to the API reference: https://api.python.langchain.com/en/latest/vectorstores/MemoryVectorStore.vectorstores.MemoryVectorStore.html